# **Connecting Google Drive to Colab**








In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **flask-ngrok is necessary to expose the port**


In [ ]:
!pip install flask-ngrok

# **Importing the required Data Science Libraries**

In [ ]:
import keras
from keras.preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np

# **Importing Flask Backend Libraries**

In [ ]:
import requests
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import json

# **Defining Constants**

In [ ]:
constants = {
    "imagePath":"/content/drive/MyDrive/(MAIN)COVID-19 Detection using X-rays/images",
    "modelPath":"/content/drive/MyDrive/(MAIN)COVID-19 Detection using X-rays/model/transferModel.h5",
    "pathToTemplate": "/content/drive/MyDrive/(MAIN)COVID-19 Detection using X-rays/templates",
    "pathToStatic": "/content/drive/MyDrive/(MAIN)COVID-19 Detection using X-rays/static",
    "apiURL": r"https://api.apify.com/v2/key-value-stores/toDWvRj1JpTXiM8FF/records/LATEST?disableRedirect=true",
}

# **Loading the Trained DL model**

In [ ]:
model = keras.models.load_model(constants["modelPath"])

# **Instantiating a flask instance and exposing it to ngrok**

In [ ]:
app = Flask(
    __name__,
    template_folder=constants["pathToTemplate"],
    static_folder=constants["pathToStatic"],
)
run_with_ngrok(app)

# **Method which acts as an Entry Point**

In [ ]:
@app.route("/", methods=["GET"])
@app.route("/index", methods=["GET"])
def index():
    response = requests.get(constants["apiURL"])
    jsonData = json.loads(response.text)
    return render_template("index.html", jsonData=jsonData)

# **The Predict Method**

In [ ]:
@app.route("/get-prediction",methods=["GET","POST"])
@app.route("/prediction",methods=["GET","POST"])
def getPrediction():
  if request.method=="GET":
    return render_template("prediction.html")
  elif request.method=="POST":
    imageFile=request.files["image"]
    imagePath=constants["imagePath"]+"/"+imageFile.filename
    imageFile.save(imagePath)
    testImage=image.load_img(imagePath,target_size=(224,224))
    x=image.img_to_array(testImage)
    x=np.expand_dims(x,axis=0)
    imageData=preprocess_input(x)
    classes=model.predict(imageData)
    prediction=np.argmax(classes,axis=1)
    if prediction==[1]:
      newPred="Normal"
    else:
      newPred="Corona"
    return render_template("prediction.html",newPred=newPred)




# **Running the app with Ngrok**

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://add565266dbe.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [11/Jul/2021 07:21:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 07:21:39] "GET /static/stylesForIndex.css HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 07:21:39] "GET /static/index.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 07:21:39] "GET /static/liquid-soap.png HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 07:21:40] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Jul/2021 07:21:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 07:21:42] "GET /static/stylesForIndex.css HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 07:21:43] "GET /static/index.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 07:21:43] "GET /static/liquid-soap.png HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 07:21:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 07:21:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 07:21:56] "GET /static/stylesForIndex.css HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 07:21:56] "GET /static/index.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 07:21:57] "GET /st